# Calculating the actual wave

## The Problem
In the **Fundamental Solution** jupyter notebook we have analytically approximated the fundamental solution. Here we are going to use that solution in order to solve the following Cauchy Global Problem.

$$
\begin{cases}

    \Delta \left( p(\vec{x},t) - \frac{1}{\omega_0}\frac{\partial}{\partial t} p(\vec{x},t)\right) - \frac{1}{c^2} \frac{\partial^2}{\partial t^2} p(\vec{x},t) = -\frac{\beta}{C_p} \frac{\partial E}{\partial t} & \forall (\vec{x},t) \in \mathbb{R}^3 \times \mathbb{R}^+\\
    p(\vec{x},0) = 0 & \forall \vec{x} \in \mathbb{R}^3\\
    \frac{\partial}{\partial t} p(\vec{x},0) = 0 & \forall \vec{x} \in \mathbb{R}^3 

\end{cases}
$$


## Notation

This notation, even though it is the most accurate is pretentious and cumbersome to work with. Therefore, from now on we will employ a different notation that is cleaner and simpler to read and write. 

First we define the linear differential operator $\mathcal{L}$ as such.

$$\mathcal{L} u(\vec{x},t) = \Delta \left( u(\vec{x},t) - \frac{1}{\omega_0}\frac{\partial}{\partial t} u(\vec{x},t)\right) - \frac{1}{c^2} \frac{\partial^2}{\partial t^2} u(\vec{x},t)$$

Now we can rewrite the cauchy problem like so:

$$
\begin{cases}

    \mathcal{L} p(\vec{x},t) = -\frac{\beta}{C_p} \frac{\partial E}{\partial t} = \psi(\vec{x},t) & \forall (\vec{x},t) \in \mathbb{R}^3 \times \mathbb{R}^+\\
    p(\vec{x},0) = 0 & \forall \vec{x} \in \mathbb{R}^3\\
    p_t(\vec{x},0) = 0 & \forall \vec{x} \in \mathbb{R}^3 

\end{cases}
$$

This is undoubtedly better. Now let's try to find the solution.

# Duhamel's Method

Duhamel's method is used in this case to simplify and solve the problem by solving the following set of problems: Let $F(\vec{x},t)$ be the *Fundamental Solution* solution defined by the following global cauchy problem:
 
 $$
\begin{cases}

    \mathcal{L} F(\vec{x},t) = \delta(\vec{x})\delta(t) & \forall (\vec{x},t) \in \mathbb{R}^3 \times \mathbb{R}^+\\
    F(\vec{x},0) = 0 & \forall \vec{x} \in \mathbb{R}^3\\
    F_t(\vec{x},0) = 0 & \forall \vec{x} \in \mathbb{R}^3 

\end{cases}
$$

Now we can use it to find the solutions $u(\vec{x},t;s)$ to the following family of problems that arise from decomposing $\psi(\vec{x},t)$ in time. So $\forall s > 0$:

$$
\begin{cases}

    \mathcal{L} u(\vec{x},t;s) = 0 & \forall (\vec{x},t) \in \mathbb{R}^3 \times (s,+\infty)\\
    u(\vec{x},s;s) = 0 & \forall \vec{x} \in \mathbb{R}^3\\
    u_t(\vec{x},s;s) = \psi(\vec{x},s) & \forall \vec{x} \in \mathbb{R}^3 

\end{cases}
$$

Therefore the solution to our original PDE will be the following convolution

$$p(\vec{x},t) = \int_0^t \int_{\mathbb{R}^3} \psi(\vec{y},s) F(\vec{x}-\vec{y},t-s) \ d\vec{y} ds$$

# Forcing Term

Now we need to find $\psi(\vec{x},t)$. To do this we need to think of the analytical form of the energy deposition of the particle in the fluid. Hence we introduce the Bethe-Bloch formula that will predict the spatial rate of energy deposition of the particle through the medium along the distance travelled from its path. 

$$ -\frac{d E}{dx} \approx - \left \langle \frac{dE}{dx} \right \rangle = \frac{4\pi}{m_e c^2}\frac{n z^2}{\beta^2}\left(\frac{e^2}{4\pi \varepsilon_0}\right)^2 \left[ \ln \left( \frac{2 m_e c^2 \beta^2}{I \cdot (1 - \beta^2)}\right) -\beta^2\right]$$

Now what we can do, is to assume that this energy is deposition will be Gaussian distributed over time and space in the medium. As a result the energy deposition in the medium is:

$$-\frac{dE}{dx} \cdot G(\vec{x},t)$$

where $G(\vec{x},t)$ is a distribution in $\vec{x}$ and $t$:

$$G(\vec{x},t) = \frac{1}{4\pi^2 \sigma_x^3 \sigma_t} \exp\left(-\frac{|\vec{x}|^2}{2\sigma_x^2}\right) \exp\left(-\frac{t^2}{2\sigma_t^2} \right)$$

Therefore we can write the forcing function $\psi(\vec{x},t)$ like so.

$$\psi(\vec{x},t) = -\frac{\beta_{Xe}}{C_p} \frac{dE}{dx} \frac{dx}{dt} G(\vec{x},t)$$

Below we plot the energy envelope.

In [3]:
# Plotting of the energy envelope

#import relevant libraries
import numpy as np
import scipy.constants as const
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import ipywidgets as widgets
from tqdm import tqdm

# if this doesn't work run: "python3 -m pip install ipympl" 
# or just comment it out and learn to live without sliders on the graphs
%matplotlib widget

s_x = 1
s_t = 1

def G(x,t,s_x=s_x,s_t=s_t):
    return 1/(4*np.pi**2 * s_x**3 * s_t)*np.exp(-np.dot(x,x)/(2*s_x**2))*np.exp(-t**2/(2*s_t**2))

In [4]:
# Plot the 2D graph
fig = plt.figure(figsize=(9,6))
fig.suptitle("Distribution of heat in space and time")
ax = fig.add_subplot(111)
ax.set_xlabel(r"Radius $|\vec{x}|$")
ax.set_ylabel(r"Time t")

x_min = 0
x_max = 3
t_min = -1
t_max = 1
Npts = 20

x = np.linspace(x_min,x_max,Npts)
t = np.linspace(t_min,t_max,Npts)

xx, tt = np.meshgrid(x,t)

def mesh_wrapper(X,T,f,params=()):
    z = np.zeros((len(X),len(T)))
    if params == ():
        for i in range(0,len(X)):
            for j in range(0,len(T)):
                z[j][i] = f(X[i],T[j])
    else:
        for i in range(0,len(X)):
            for j in range(0,len(T)):
                z[j][i] = f(X[i],T[j],*params)

    return z 

zz = mesh_wrapper(x,t,G)


ax.pcolormesh(xx,tt,zz, shading='gouraud', cmap=plt.cm.hot)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…

# Integrating

Now to do this integral we need to first express it analytically. This should be fun!

$$
p(\vec{x},t) = \int_0^t \int_{\mathbb{R}^3} -\frac{\beta_{Xe}}{C_p} \frac{4\pi}{m_e c^2}\frac{n z^2}{\beta^2}\left(\frac{e^2}{4\pi \varepsilon_0}\right)^2 \left[ \ln \left( \frac{2 m_e c^2 \beta^2}{I \cdot (1 - \beta^2)}\right) -\beta^2\right] \frac{dz}{dt} \frac{1}{4\pi^2 \sigma_x^3 \sigma_t} e^{-\frac{|\vec{y}|^2}{2\sigma_x^2}} e^{-\frac{s^2}{2\sigma_t^2}} 

\Theta(t-s) \frac{8\pi^2 c}{|\vec{x}-\vec{y}|(t-s)}\sqrt{\frac{\pi \omega_0}{8 c^2 (t-s)}} \exp \left(-\frac{|\vec{x}-\vec{y}|^2 \omega_0}{c^2(t-s)}-(t-s)\omega_0\right) \left[\exp\left(\frac{(|\vec{x}-\vec{y}|+c(t-s))^2\omega_0}{2c^2(t-s)}\right)-\exp\left(\frac{(|\vec{x}-\vec{y}|-c(t-s))^2\omega_0}{2c^2(t-s)}\right)\right]

 \ d\vec{y} ds
$$

cool...

This keeps happening. But I think we can simplify the integral to a much better form. Frist of all, assuming the velocity of the particle to be constantnt we can reqwrite everything like this.


$$
p(\vec{x},t) = \mathcal{A} \int_0^t \int_{\mathbb{R}^3} 

\frac{1}{|\vec{x}-\vec{y}|(t-s)^{\frac{3}{2}}} \exp \left(-\frac{|\vec{y}|^2}{2\sigma_x^2} -\frac{s^2}{2\sigma_t^2} -\frac{|\vec{x}-\vec{y}|^2 \omega_0}{c^2(t-s)}-(t-s)\omega_0\right) \left[\exp\left(\frac{(|\vec{x}-\vec{y}|+c(t-s))^2\omega_0}{2c^2(t-s)}\right)-\exp\left(\frac{(|\vec{x}-\vec{y}|-c(t-s))^2\omega_0}{2c^2(t-s)}\right)\right]

 \ d\vec{y} ds
$$
